<a href="https://colab.research.google.com/github/sametgumus212/java-kodlar-2/blob/master/f_p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-binance
!pip install catboost
!pip install matplotlib --upgrade
!pip install mpl_finance

In [3]:
import pandas as pd
import numpy as np

import time
import dateparser
import pytz
import json

import math  

import datetime as dt
from datetime import datetime, timedelta

from threading import Thread

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates

from mpl_finance import candlestick_ochl



from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error

from catboost import CatBoostClassifier, CatBoostRegressor, Pool, cv

from tqdm import tqdm as tqdm

from binance.client import Client

def financial indicators 

MA = moving average

TP = Typical price

EMA= exponential moving Average


In [4]:
def moving_average(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))
  df = df.join(MA)
  return df

In [5]:
def exponential_moving_average(df, n):
  EMA = pd.Series(df['Close'].ewm(span=n, min_periods=n).mean(), name="EMA_" + str(n))
  df = df.join(EMA)
  return df

In [6]:
def momentum(df, n):
  M = pd.Series(df['Close'].diff(n), name="Momentum_"+str(n))
  df = df.join(M)
  return df

In [7]:
def bollinger_bands(df, n):
  MA = pd.Series(df['Close'].rolling(n, min_periods=n).mean())
  MSD = pd.Series(df['Close'].rolling(n, min_periods=n).std())  
  b1 = 4 * MSD / MA
  B1 = pd.Series(b1, name = "BollingerB_" + str(n))
  df = df.join(B1)
  b2 = (df['Close'] - MA  + 2 * MSD) / (4 * MSD)
  B2 = pd.Series(b2, name='Bollinger%b_'+str(n))
  df = df.join(B2)
  return df

In [8]:
def stochastic_oscillator(df ,n):
  S0K = pd.Series((df['Close']- df['Low']) / (df['High']-df['Low']), name='S0k_' + str(n))
  df = df.join(S0K)
  return df

In [9]:
def relative_strength_index(df, n):
  i = 0
  UpI = [0]
  DoI = [0]

  while i + 1 <= df.index[-1]:
    UpMove = df.loc[i+1, 'High'] - df.loc[i, 'High']
    DoMove = df.loc[i+1, 'Low'] - df.loc[i, 'Low']
    if UpMove > DoMove and UpMove > 0:
      UpD = UpMove
    else:
      UpD = 0
    UpI.append(UpD)
    if DoMove > UpMove and DoMove > 0:
      DoD = DoMove
    else:
      DoD = 0
    DoI.append(DoD)
    i = i + 1

  UpI = pd.Series(UpI)
  DoI = pd.Series(DoI)
  PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean())
  NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean())
  RSI = pd.Series(PosDI / (PosDI + NegDI), name='RSI_' + str(n))
  df = df.join(RSI)
  return df

In [10]:
def common_channel_index(df, n):
  PP = (df['High'] + df['Low'] + df['Close']) / 3
  CCI = pd.Series((PP - PP.rolling(n, min_periods=n).mean()) / PP.rolling(n, min_periods=n).std(), name='CCI_' + str(n))
  df = df.join(CCI)
  return df

In [11]:
def standart_deviation(df, n):
  df = df.join(pd.Series(df['Close'].rolling(n, min_periods=n).std(), name='STD_' + str(n)))
  return df

binance helper function

In [12]:
def binanceDataFrame(klines):
  df = pd.DataFrame(klines.reshape(-1,12), dtype=float, columns=['Open Time',
                                                                 'Open',
                                                                 'High',
                                                                 'Low',
                                                                 'Close',
                                                                 'Volume',
                                                                 'Close Time',
                                                                 'Quote asset volume',
                                                                 'Number of trades',
                                                                 'Taker buy base asset volume',
                                                                 'Taker buy quote asset volume',
                                                                 'Can be ignored']);

  df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
  df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')

  return df

In [13]:
def date_to_miliseconds(date_str):
  epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
  d = dateparser.parse(date_str)
  if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
    d = d.replace(tzinfo=pytz.utc)

  #return difference in time
  return int((d - epoch).total_seconds()*1000.0)

In [14]:
def interval_to_miliseconds(interval):
  ms = None
  seconds_per_unit = {
      "m": 60,
      "h": 60 * 60,
      "d": 24 * 60 * 60,
      "w": 7 * 24 * 60 * 60,
  }

  unit = interval[-1]
  if unit in seconds_per_unit:
    try:
      ms = int(interval[:-1]) * seconds_per_unit[unit] * 1000
    except:
      pass

  return ms

In [15]:
def get_historical_klines(symbol, interval, start_str, end_str=None):
  output_data=[]
  limit=500
  timeFrame = interval_to_miliseconds(interval)
  start_ts = date_to_miliseconds(start_str)
  end_ts = None
  if end_str:
    end_ts = date_to_miliseconds(end_str)
  
  idx = 0
  symbol_existed = False
  while True:
    #fetch
    temp_data = client.get_klines(symbol = symbol, 
                                  interval = interval, 
                                  limit = limit, 
                                  startTime = start_ts, 
                                  endTime = end_ts)
    
    if not symbol_existed and len(temp_data):
      symbol_existed = True

    if symbol_existed:
      output_data += temp_data

      start_ts = temp_data[len(temp_data) - 1][0] + timeFrame

    idx += 1

    if len(temp_data)<limit:
      break

    if idx % 3 == 0:
      time.sleep(1)

  return output_data

binance client

In [16]:
api_key = 'MF9wOZEQgDTnHqD8AalLHapsFA26pdEsPgPZ4hCAl3U1935aLTXnALARzppIuJ7S'  # MF9wOZEQgDTnHqD8AalLHapsFA26pdEsPgPZ4hCAl3U1935aLTXnALARzppIuJ7S
api_secret = 'omyizbSVVZIEBW6GiYxhjVFbjTzTYJvItelDvXKBF0E2bedcuOmRSkUZB942rLIp'  #omyizbSVVZIEBW6GiYxhjVFbjTzTYJvItelDvXKBF0E2bedcuOmRSkUZB942rLIp
# api_key = ''
# api_secret = ''
client = Client(api_key, api_secret)

In [17]:
depth = client.get_order_book(symbol = 'BTCUSDT')

In [18]:
depth

{'asks': [['48995.02000000', '1.16213700'],
  ['48999.51000000', '0.59150700'],
  ['48999.52000000', '0.89300000'],
  ['48999.66000000', '0.02139300'],
  ['49000.35000000', '0.79700000'],
  ['49000.61000000', '0.47900000'],
  ['49001.20000000', '0.94700000'],
  ['49001.91000000', '0.20207800'],
  ['49001.92000000', '1.00000000'],
  ['49002.36000000', '0.68234200'],
  ['49003.59000000', '0.01000000'],
  ['49003.63000000', '0.21000000'],
  ['49004.67000000', '0.22140300'],
  ['49004.68000000', '0.51413600'],
  ['49004.80000000', '0.04006900'],
  ['49005.34000000', '0.10000000'],
  ['49006.06000000', '0.03000000'],
  ['49007.61000000', '0.00101200'],
  ['49007.70000000', '0.02000000'],
  ['49009.41000000', '0.13353800'],
  ['49009.96000000', '0.10197000'],
  ['49010.00000000', '0.01633900'],
  ['49010.15000000', '0.02300000'],
  ['49010.83000000', '0.11500000'],
  ['49011.79000000', '0.00806500'],
  ['49011.80000000', '0.00442200'],
  ['49012.06000000', '0.05531100'],
  ['49012.07000000',

In [19]:
%%time
symbol = "BTCUSDT"
start = "1 Jan, 2017"
interval = Client.KLINE_INTERVAL_1DAY
klines = get_historical_klines(symbol,interval,start)
klines = np.array(klines)
df = binanceDataFrame(klines)

CPU times: user 97 ms, sys: 20.8 ms, total: 118 ms
Wall time: 467 ms


In [20]:
df

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Can be ignored
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,2017-08-17 23:59:59.999,3.454770e+06,3427.0,616.248541,2.678216e+06,8733.911395
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,2017-08-18 23:59:59.999,5.086958e+06,5233.0,972.868710,4.129123e+06,9384.141409
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,2017-08-19 23:59:59.999,1.549484e+06,2153.0,274.336042,1.118002e+06,9184.085529
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,2017-08-20 23:59:59.999,1.930364e+06,2321.0,376.795947,1.557401e+06,10125.414084
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2017-08-21 23:59:59.999,2.797232e+06,3972.0,557.356107,2.255663e+06,11706.769970
...,...,...,...,...,...,...,...,...,...,...,...,...
1461,2021-08-17,45901.30,47160.00,44376.00,44695.95,57039.341629,2021-08-17 23:59:59.999,2.616340e+09,1781303.0,27320.261808,1.253907e+09,0.000000
1462,2021-08-18,44695.95,46000.00,44203.28,44705.29,54099.415985,2021-08-18 23:59:59.999,2.439673e+09,1656723.0,28538.223727,1.286718e+09,0.000000
1463,2021-08-19,44699.37,47033.00,43927.70,46760.62,53411.753920,2021-08-19 23:59:59.999,2.416320e+09,1774382.0,27193.505150,1.230828e+09,0.000000
1464,2021-08-20,46760.62,49382.99,46622.99,49322.47,56850.352228,2021-08-20 23:59:59.999,2.721132e+09,1950404.0,29725.885112,1.422945e+09,0.000000


In [21]:
df.describe()

,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Can be ignored
count,1466.000000,1466.000000,1466.000000,1466.000000,1466.000000,1.466000e+03,1.466000e+03,1466.000000,1.466000e+03,1466.000000
mean,14190.408588,14646.064338,13674.176460,14221.001228,50775.070098,9.176029e+08,6.941440e+05,25313.212070,4.538283e+08,1720.836325
std,13996.707090,14477.860964,13442.192212,14023.881689,37590.767598,1.371050e+09,7.492290e+05,18259.530530,6.715867e+08,7186.002612
min,3188.010000,3276.500000,2817.000000,3189.020000,228.108068,9.778657e+05,2.153000e+03,56.190141,2.413638e+05,0.000000
25%,6590.512500,6716.700000,6455.750000,6593.910000,27564.257705,1.983111e+08,2.019108e+05,14183.053579,1.031068e+08,0.000000
50%,8901.675000,9181.920000,8655.810000,8905.285000,43473.899234,3.658690e+08,3.929375e+05,21699.752864,1.852541e+08,0.000000
75%,11816.252500,12049.557500,11456.000000,11826.815000,66563.360479,8.072771e+08,8.826118e+05,33100.705929,3.947308e+08,0.000000
max,63575.010000,64854.000000,62020.000000,63575.000000,402201.673764,1.347769e+10,6.331062e+06,197511.954273,6.410033e+09,76007.970079


preprocessing

In [22]:
remove_cols = [c for c in df.columns if c not in ['Open','Close','High','Low','Volume']]
df.drop(remove_cols, axis=1, inplace=True)

calculate financial indicators for several windows

In [23]:
windows = [3,5,9,14,30,50,100,150,300]

In [24]:
%%time

for w in tqdm(windows):
  df = moving_average(df, w)
  df = exponential_moving_average(df, w)
  df = momentum(df,w)
  df = bollinger_bands(df,w)
  df = stochastic_oscillator(df,w)
  df = relative_strength_index(df,w)
  df = common_channel_index(df,w)
  df = standart_deviation(df,w)

100%|██████████| 9/9 [00:00<00:00, 10.49it/s]

CPU times: user 810 ms, sys: 19.8 ms, total: 830 ms
Wall time: 866 ms


In [25]:
df

,Open,High,Low,Close,Volume,MA_3,EMA_3,Momentum_3,BollingerB_3,Bollinger%b_3,S0k_3,RSI_3,CCI_3,STD_3,MA_5,EMA_5,Momentum_5,BollingerB_5,Bollinger%b_5,S0k_5,RSI_5,CCI_5,STD_5,MA_9,EMA_9,Momentum_9,BollingerB_9,Bollinger%b_9,S0k_9,RSI_9,CCI_9,STD_9,MA_14,EMA_14,Momentum_14,BollingerB_14,Bollinger%b_14,S0k_14,RSI_14,CCI_14,...,S0k_30,RSI_30,CCI_30,STD_30,MA_50,EMA_50,Momentum_50,BollingerB_50,Bollinger%b_50,S0k_50,RSI_50,CCI_50,STD_50,MA_100,EMA_100,Momentum_100,BollingerB_100,Bollinger%b_100,S0k_100,RSI_100,CCI_100,STD_100,MA_150,EMA_150,Momentum_150,BollingerB_150,Bollinger%b_150,S0k_150,RSI_150,CCI_150,STD_150,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,STD_300
0,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,...,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.296294,NaN,NaN,NaN
1,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,...,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.391912,NaN,NaN,NaN
2,4108.37,4184.69,3850.00,4139.98,381.309763,4177.810000,4151.677143,NaN,0.090223,0.399638,0.866414,NaN,-0.849879,94.233421,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,...,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.866414,NaN,NaN,NaN
3,4120.98,4211.08,4032.62,4086.29,467.083022,4111.546667,4116.804000,-198.79,0.026253,0.266017,0.300740,0.000000,0.179888,26.985597,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,...,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.300740,NaN,NaN,NaN
4,4069.13,4119.62,3911.79,4016.00,691.743060,4080.756667,4064.776129,-92.37,0.060945,0.239619,0.501419,0.000000,-0.965315,62.174942,4127.144,4086.052607,NaN,0.096291,0.220327,0.501419,0.000000,-0.958744,99.351704,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,...,0.501419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501419,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,45901.30,47160.00,44376.00,44695.95,57039.341629,45857.020000,45533.034871,-2372.56,0.099403,0.245286,0.114925,0.642353,-1.152818,1139.580102,46487.914,45735.753437,296.95,0.104056,0.129558,0.114925,0.515745,-1.687665,1209.341631,46020.884444,45364.268716,901.58,0.097541,0.204843,0.114925,0.423173,-0.715517,1122.228280,44712.670000,44434.196188,6488.90,0.206964,0.498193,0.114925,0.394786,0.371749,...,0.114925,0.398722,0.989662,5248.437633,37630.8234,40148.639368,10201.06,0.568694,0.830139,0.114925,0.403500,1.491768,5350.107689,38114.5583,40255.454265,-13544.89,0.580209,0.797606,0.114925,0.401886,1.318666,5528.604887,44316.594933,40296.636355,-13406.33,0.907731,0.509430,0.114925,0.407147,0.114631,10056.892043,38072.994867,36460.496448,31914.99,1.470432,0.618301,0.114925,0.425031,0.534091,13995.937977
1462,4

In [26]:
df.describe()

,Open,High,Low,Close,Volume,MA_3,EMA_3,Momentum_3,BollingerB_3,Bollinger%b_3,S0k_3,RSI_3,CCI_3,STD_3,MA_5,EMA_5,Momentum_5,BollingerB_5,Bollinger%b_5,S0k_5,RSI_5,CCI_5,STD_5,MA_9,EMA_9,Momentum_9,BollingerB_9,Bollinger%b_9,S0k_9,RSI_9,CCI_9,STD_9,MA_14,EMA_14,Momentum_14,BollingerB_14,Bollinger%b_14,S0k_14,RSI_14,CCI_14,...,S0k_30,RSI_30,CCI_30,STD_30,MA_50,EMA_50,Momentum_50,BollingerB_50,Bollinger%b_50,S0k_50,RSI_50,CCI_50,STD_50,MA_100,EMA_100,Momentum_100,BollingerB_100,Bollinger%b_100,S0k_100,RSI_100,CCI_100,STD_100,MA_150,EMA_150,Momentum_150,BollingerB_150,Bollinger%b_150,S0k_150,RSI_150,CCI_150,STD_150,MA_300,EMA_300,Momentum_300,BollingerB_300,Bollinger%b_300,S0k_300,RSI_300,CCI_300,STD_300
count,1466.000000,1466.000000,1466.000000,1466.000000,1466.000000,1464.000000,1464.000000,1463.000000,1464.000000,1464.000000,1466.000000,1463.000000,1464.000000,1464.000000,1462.000000,1462.000000,1461.000000,1462.000000,1462.000000,1466.000000,1462.000000,1462.000000,1462.000000,1458.000000,1458.000000,1457.000000,1458.000000,1458.000000,1466.000000,1458.000000,1458.000000,1458.000000,1453.000000,1453.000000,1452.000000,1453.000000,1453.000000,1466.000000,1453.000000,1453.000000,...,1466.000000,1437.000000,1437.000000,1437.000000,1417.000000,1417.000000,1416.000000,1417.000000,1417.000000,1466.000000,1417.000000,1417.000000,1417.000000,1367.000000,1367.000000,1366.000000,1367.000000,1367.000000,1466.000000,1367.000000,1367.000000,1367.000000,1317.000000,1317.000000,1316.000000,1317.000000,1317.000000,1466.000000,1317.000000,1317.000000,1317.000000,1167.000000,1167.000000,1166.000000,1167.000000,1167.000000,1466.000000,1167.000000,1167.000000,1167.000000
mean,14190.408588,14646.064338,13674.176460,14221.001228,50775.070098,14204.000997,14204.534392,90.597861,0.104852,0.518885,0.559001,0.433846,0.084173,403.057031,14188.226175,14188.980195,146.367981,0.138016,0.528015,0.559001,0.450279,0.123560,527.978847,14158.210658,14159.279486,264.129403,0.185565,0.538045,0.559001,0.465082,0.163250,703.569119,14120.083538,14124.483000,405.098168,0.233914,0.547281,0.559001,0.473886,0.197934,...,0.559001,0.486263,0.267036,1316.693801,13917.983791,13930.419881,1218.303234,0.481496,0.573328,0.559001,0.493609,0.297305,1748.908554,13684.317179,13624.837016,2388.275337,0.733651,0.569473,0.559001,0.498887,0.278200,2712.739609,13222.315010,13192.363757,4089.115137,0.908232,0.557449,0.559001,0.498570,0.230181,3333.521686,11410.461503,12071.404920,7705.202376,1.321873,0.636132,0.559001,0.502489,0.544173,4532.815336
std,13996.707090,14477.860964,13442.192212,14023.881689,37590.767598,13989.618110,13984.780555,1482.971251,0.091903,0.213928,0.258266,0.303990,0.892206,574.444692,13960.420949,13950.248973,1966.682262,0.099866,0.257242,0.258266,0.243311,1.075510,677.250001,13904.592917,13883.648487,2702.964653,0.122408,0.295216,0.258266,0.185784,1.221705,884.263912,13832.754590,13805.611710,3445.600350,0.147509,0.317347,0.258266,0.153311,1.302799,...,0.258266,0.111642,1.405799,1592.006445,13456.610434,13288.375169,6946.508580,0.262483,0.361000,0.258266,0.088196,1.451798,2000.651916,12958.249763,12416.475901,10707.744945,0.391619,0.378238,0.258266,0.066622,1.518154,3053.469663,12090.611930,11508.108042,12041.138054,0.450260,0.372123,0.258266,0.059208,1.491422,3849.888843,8230.555422,9287.248517,14831.600495,0.613628,0.384084,0.258266,0.045203,1.538285,5305.655454
min,3188.010000,3276.500000,2817.000000,3189.020000,228.108068,3221.520000,3241.400929,-9741.410000,0.000794,0.211325,0.000225,0.000000,-1.154701,0.869042,3282.324000,3278.295930,-13151.360000,0.005664,0.053086,0.000225,0.000000,-1.784676,5.404293,3353.465556,3372.283424,-19126.050000,0.018358,-0.144555,0.000225,0.011810,-2.581623,20.237727,3422.697857,3472.761187,-23585.590000,0.030003,-0.324226,0.000225,0.041329,-3.242502,...,0.000225,0.144767,-3.975435,76.488670,3619.334600,3678.001925,-26330.980000,0.073495,-0.688138,0.000225,0.233588,-4.390835,119.877378,3678.423400,4013.722189,-31287.170000,0.1480

define Train Data

In [27]:
X = df.drop(['Close', 'High', 'Low', 'Volume'], axis=1)
y = df['Close']

In [28]:
X = X[max(windows)+1:]
y = y[max(windows)+1:]
X.fillna(value=0, inplace=True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [29]:
%%time
scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns=X.columns)

CPU times: user 6.96 ms, sys: 0 ns, total: 6.96 ms
Wall time: 7.99 ms


split data

In [30]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, shuffle = False)
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X_train, y_train, test_size=0.2, shuffle = True)
print("Train", X_train.shape, y_train.shape)
print("Valid", X_valid.shape, y_valid.shape)
print("Test", X_test.shape, y_test.shape)

Train (745, 82) (745,)
Valid (187, 82) (187,)
Test (233, 82) (233,)


Train

In [31]:
# https://catboost.ai/docs/search/?query=catboostregressor
model = CatBoostRegressor(iterations=20_000,
                          verbose=100, 
                          # boosting_type = 'Ordered',
                          early_stopping_rounds=200,
                          loss_function = 'RMSE',
                          custom_metric = 'MAE',
                          task_type = 'CPU'
                          )

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model = True)

Learning rate set to 0.006742
0:	learn: 3822.1804909	test: 3859.7296019	best: 3859.7296019 (0)	total: 72ms	remaining: 24m
100:	learn: 2234.9392811	test: 2355.8167800	best: 2355.8167800 (100)	total: 2.06s	remaining: 6m 45s
200:	learn: 1335.4025702	test: 1510.2595148	best: 1510.2595148 (200)	total: 4.02s	remaining: 6m 35s
300:	learn: 819.6923699	test: 1023.5545747	best: 1023.5545747 (300)	total: 5.98s	remaining: 6m 31s
400:	learn: 526.9741385	test: 745.0205652	best: 745.0205652 (400)	total: 7.95s	remaining: 6m 28s
500:	learn: 357.5617605	test: 573.0212656	best: 573.0212656 (500)	total: 9.89s	remaining: 6m 24s
600:	learn: 259.0069788	test: 466.5329070	best: 466.5329070 (600)	total: 11.8s	remaining: 6m 22s
700:	learn: 198.9136613	test: 394.8891967	best: 394.8891967 (700)	total: 13.8s	remaining: 6m 19s
800:	learn: 162.2871973	test: 346.3009797	best: 346.3009797 (800)	total: 15.8s	remaining: 6m 17s
900:	learn: 138.3705770	test: 313.9289268	best: 313.9289268 (900)	total: 17.7s	remaining: 6m 1

hyper-parameter tuning

In [32]:
model_hyp = CatBoostRegressor(                          
                          early_stopping_rounds=100,
                          loss_function = 'RMSE',
                          custom_metric = 'MAE',
                          task_type = 'GPU'
                          )
grid = {
    'boosting_type': ['Ordered', 'Plain'],
    'learning_rate': [0.003, 0.006, 0.009, 0.03, 0.1, 0.5],
    'depth': [4, 6, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

In [33]:
X_train_cv = pd.concat([X_train, X_valid])
y_train_cv = pd.concat([y_train, y_valid])
X_train_cv = X_train_cv[:360]
y_train_cv = y_train_cv[:360]

In [34]:
print("Train", X_train_cv.shape, y_train_cv.shape)

Train (360, 82) (360,)


In [35]:
grid_search_results = model_hyp.grid_search(grid, X_train_cv, y_train_cv, verbose=True)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
526:	learn: 3243.5477147	test: 4329.3224515	best: 4329.3224515 (526)	total: 7.86s	remaining: 7.06s
527:	learn: 3237.7160674	test: 4322.8681837	best: 4322.8681837 (527)	total: 7.88s	remaining: 7.04s
528:	learn: 3231.6698126	test: 4316.5319413	best: 4316.5319413 (528)	total: 7.89s	remaining: 7.03s
529:	learn: 3224.6996759	test: 4309.3683992	best: 4309.3683992 (529)	total: 7.91s	remaining: 7.01s
530:	learn: 3218.8695048	test: 4303.0661936	best: 4303.0661936 (530)	total: 7.92s	remaining: 7s
531:	learn: 3212.6479214	test: 4296.4032503	best: 4296.4032503 (531)	total: 7.93s	remaining: 6.97s
532:	learn: 3206.5184997	test: 4291.4832453	best: 4291.4832453 (532)	total: 7.94s	remaining: 6.96s
533:	learn: 3200.2738772	test: 4284.6695192	best: 4284.6695192 (533)	total: 7.96s	remaining: 6.94s
534:	learn: 3193.5771654	test: 4277.4102237	best: 4277.4102237 (534)	total: 7.97s	remaining: 6.93s
535:	learn: 3188.5941172	test: 4271.7523857	best: 4271.7523857 (5

In [36]:
grid_search_results

{'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
             

use hyper parameters

In [37]:
model1 = CatBoostRegressor(                          
    early_stopping_rounds=100,
    verbose=100, 
    boosting_type = 'Plain',
    depth =  4,
    l2_leaf_reg =  7,
    learning_rate =  0.03,
    loss_function = 'RMSE',
    custom_metric = 'MAE',
    task_type = 'GPU'
    )

model1.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model = True, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 3748.1804459	test: 3789.7716777	best: 3789.7716777 (0)	total: 5.86ms	remaining: 5.85s
100:	learn: 520.9096844	test: 798.4722846	best: 798.4722846 (100)	total: 477ms	remaining: 4.24s
200:	learn: 207.4608656	test: 443.5659583	best: 443.5659583 (200)	total: 984ms	remaining: 3.91s
300:	learn: 128.6696514	test: 336.5921062	best: 336.5921062 (300)	total: 1.47s	remaining: 3.4s
400:	learn: 99.6430104	test: 298.6683278	best: 298.6683278 (400)	total: 1.98s	remaining: 2.96s
500:	learn: 85.5632503	test: 278.6415672	best: 278.6415672 (500)	total: 2.49s	remaining: 2.48s
600:	learn: 76.3226417	test: 269.4831952	best: 269.4831952 (600)	total: 3.04s	remaining: 2.02s
700:	learn: 69.3835802	test: 265.4894179	best: 265.4894179 (700)	total: 3.52s	remaining: 1.5s
800:	learn: 63.6826734	test: 263.6465995	best: 263.6206154 (799)	total: 4.04s	remaining: 1s
900:	learn: 58.0977307	test: 261.8678689	best: 261.8678689 (900)	total: 4.56s	remaining: 501ms
999:	learn: 53.7034237	test: 260.3638035	best: 260.

save and load models

In [38]:
model.save_model('base_model_CPU.model')

In [39]:
model1.save_model('hyp_model1_GPU.model')

test

In [40]:
model_use = CatBoostRegressor()
model_use.load_model('hyp_model1_GPU.model')

In [41]:
%%time
y_pred = model_use.predict(X_train)
mse_train = mean_squared_error(y_train,y_pred)
print('mean_squared_error train', math.sqrt(mse_train))
mae_train = mean_absolute_error(y_train,y_pred)
print('mean_absolute_error train', mae_train)

y_pred = model_use.predict(X_valid)
mse_valid = mean_squared_error(y_valid,y_pred)
print('mean_squared_error valid', math.sqrt(mse_valid))
mae_valid = mean_absolute_error(y_valid,y_pred)
print('mean_absolute_error valid', mae_valid)

y_pred = model_use.predict(X_test)
mse_test = mean_squared_error(y_test,y_pred)
print('mean_squared_error test', math.sqrt(mse_test))
mae_test = mean_absolute_error(y_test,y_pred)
print('mean_absolute_error test', mae_test)

mean_squared_error train 53.703264505879694
mean_absolute_error train 40.48141165548357
mean_squared_error valid 260.36315687420125
mean_absolute_error valid 103.07458346195678
mean_squared_error test 23984.752169712676
mean_absolute_error test 22265.06873078825
CPU times: user 49.8 ms, sys: 6.16 ms, total: 55.9 ms
Wall time: 41 ms


strategy Tester

In [42]:
%%time 
X_test_unscaled = scaler.inverse_transform(X_test)
X_test_unscaled = pd.DataFrame(X_test_unscaled, columns=X_test.columns)

CPU times: user 470 µs, sys: 0 ns, total: 470 µs
Wall time: 477 µs


In [43]:
X_test_unscaled['Pred_Close'] = y_pred
X_test_unscaled['Close'] = y_test.values

In [44]:
X_test_unscaled = X_test_unscaled[['Open', 'Close', 'Pred_Close']]

In [45]:
X_test_unscaled['Diff'] = X_test_unscaled['Close'] - X_test_unscaled['Open']
X_test_unscaled['Change'] = X_test_unscaled['Diff'] / X_test_unscaled['Open']
X_test_unscaled['Buy'] = X_test_unscaled['Pred_Close'] > X_test_unscaled['Close']

In [46]:
# Bitcoin alınması gereken günler
X_test_unscaled[X_test_unscaled['Buy'] == True]

,Open,Close,Pred_Close,Diff,Change,Buy


In [47]:
# Bitcoin satılması gereken günler
X_test_unscaled[X_test_unscaled['Buy'] == False]

,Open,Close,Pred_Close,Diff,Change,Buy
0,28923.63,29331.69,26601.484988,408.06,0.014108,False
1,29331.70,32178.33,26399.583339,2846.63,0.097050,False
2,32176.45,33000.05,25793.260688,823.60,0.025596,False
3,33000.05,31988.71,25690.188859,-1011.34,-0.030647,False
4,31989.75,33949.53,25878.175484,1959.78,0.061263,False
...,...,...,...,...,...,...
228,45901.30,44695.95,20810.459285,-1205.35,-0.026260,False
229,44695.95,44705.29,20481.990622,9.34,0.000209,False
230,44699.37,46760.62,21971.016439,2061.25,0.046114,False
231,46760.62,49322.47,22785.072474,2561.85,0.054786,False


plot Accuracy

In [48]:
#https://matplotlib.org/

figure(num=None, figsize = (12, 10), dpi=80, facecolor='silver', edgecolor='gray')

plt.subplot(2, 1, 1)
plt.plot(X_test_unscaled[['Close','Pred_Close']])
plt.xlabel('time (d1)')
plt.ylabel('$')
plt.legend(['Close', 'Prediction'])
plt.title('Close vs Predicted Close')
plt.grid(True)

plt.tight_layout()
plt.show()

ImportError: ignored

<Figure size 960x800 with 1 Axes>

simulation

In [49]:
Init_Balance = 10_000
Ratio = 1
Balance = Init_Balance
BalanceGraph = []
last_order = False
SellCount = 0
BuyCount = 0
AccurateMove = 0
GrossLoss = 0
GrossGain = 0
CommisionRatio = 0
Commision = 0 

In [50]:
%%time
for index, row in tqdm(X_test_unscaled.iterrows()):
  buy = row.Buy
  change = row.Change

  if (buy):
    Balance += change * Balance * Ratio
    BuyCount += 1
    if (change > 0):
      AccurateMove += 1
      GrossGain += abs(change * Balance * Ratio)
    else:
      GrossLoss += abs(change * Balance * Ratio)
  else:
    Balance += -change * Balance * Ratio
    SellCount += 1
    if (change < 0):
      AccurateMove += 1
      GrossGain += abs(change * Balance * Ratio)
    else:
      GrossLoss += abs(change * Balance * Ratio)

  if (last_order != buy):
    Balance -= CommisionRatio * abs(change * Balance * Ratio)
    Commision += CommisionRatio * abs(change * Balance * Ratio)

  last_order = buy

  BalanceGraph.append(Balance)

233it [00:00, 5958.29it/s]

CPU times: user 45.9 ms, sys: 32.5 ms, total: 78.5 ms
Wall time: 50.3 ms


In [51]:
print("Balance", Balance)
print(f"Sell:{round(SellCount)}, Buy:{round(BuyCount)}, Sell Ratio: {(SellCount / (SellCount + BuyCount))}, Buy Ratio: {(BuyCount / (SellCount + BuyCount))}")
print(f"Accuracy: {AccurateMove / (SellCount + BuyCount)}")
print(f"Gross Loss:{round(GrossLoss)}, GrossGain:{round(GrossGain)}")

Balance 3571.711088679454
Sell:233, Buy:0, Sell Ratio: 1.0, Buy Ratio: 0.0
Accuracy: 0.4978540772532189
Gross Loss:23463, GrossGain:19882


In [52]:
figure(num=None, figsize = (12, 8), dpi=80, facecolor='silver', edgecolor='gray')

plt.subplot(2, 1, 1)
plt.plot(X_test_unscaled[['Close','Pred_Close']])
plt.xlabel('time (d1)')
plt.ylabel('$')
plt.legend(['Close', 'Prediction'])
plt.title('Close vs Predicted Close')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(BalanceGraph)
plt.xlabel('time (d1)')
plt.ylabel('$')
plt.legend(['Balance'])
plt.title('Balance')
plt.grid(True)

plt.tight_layout()
plt.show()

ImportError: ignored

<Figure size 960x640 with 2 Axes>